In [34]:
import gensim
from gensim import corpora
import math

class BM25 :
    def __init__(self, fn_docs, raw_data,delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        '''
     For reading files and converting to data and add in documents
         raw_data = []
         for line in file(self.fn_docs) :
             raw_data.append(line.strip().split(self.delimiter))
        '''
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        for doc in raw_data :
#             doc = line.strip().split(self.delimiter) -- for converting from file 
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items

if __name__ == '__main__' :
    #mycorpus.txt is as following:
    '''
    Human machine interface for lab abc computer applications
    A survey of user opinion of computer system response time
    The EPS user interface management system
    System and human system engineering testing of EPS
    Relation of user perceived response time to error measurement
    The generation of random binary unordered trees
    The intersection graph of paths in trees
    Graph IV Widths of trees and well quasi ordering
    Graph minors A survey
    '''
    fn_docs = 'mycorpus.txt'
    raw_data = [['Human', 'machine', 'interface', 'for', 'lab', 'abc', 'computer', 'applications'],
                ['A', 'survey', 'of', 'user', 'opinion', 'of', 'computer', 'system', 'response', 'time'], 
                ['The', 'EPS', 'user', 'interface', 'management', 'system'], 
                ['System', 'and', 'human', 'system', 'engineering', 'testing', 'of', 'EPS'], 
                ['Relation', 'of', 'user', 'perceived', 'response', 'time', 'to', 'error', 'measurement'], 
                ['The', 'generation', 'of', 'random', 'binary', 'unordered', 'trees'], 
                ['The', 'intersection', 'graph', 'of', 'paths', 'in', 'trees'], 
                ['Graph', 'IV', 'Widths', 'of', 'trees', 'and', 'well', 'quasi', 'ordering'],
                ['Graph', 'minors', 'A', 'survey']]
    bm25 = BM25(fn_docs, raw_data, delimiter=' ')
    print bm25
    Query = 'What is a Graph ?'
    Query = Query.split()
    scores = bm25.BM25Score(Query)
    #print "Len of scores",len(scores)
    print "scores",scores
    tfidf = bm25.TFIDF()
    #print bm25.Items()
    for i, tfidfscore in enumerate(tfidf):
        print i, tfidfscore
                                 
'''
    Human machine interface for lab abc computer applications
    A survey of user opinion of computer system response time
'''

<__main__.BM25 instance at 0x10a328f80>
scores [0, 0, 0, 0, 0, 0, 0, 0.15791141726235663, 0.5415694380758288]
0 [(0, 0.2168251319235133), (1, 0.2168251319235133), (2, 0.2168251319235133), (3, 0.2168251319235133), (4, 0.2168251319235133), (5, 0.13732653608351372), (6, 0.2168251319235133), (7, 0.13732653608351372)]
1 [(5, 0.10986122886681099), (8, 0.10986122886681099), (9, -0.1238078416812447), (10, 0.06190392084062235), (11, 0.10986122886681099), (12, 0.06190392084062235), (13, 0.10986122886681099), (14, 0.17346010553881064), (15, 0.10986122886681099)]
2 [(7, 0.1831020481113516), (10, 0.10317320140103725), (12, 0.10317320140103725), (16, 0.10317320140103725), (17, 0.28910017589801773), (18, 0.1831020481113516)]
3 [(9, -0.07737990105077794), (10, 0.07737990105077794), (18, 0.13732653608351372), (19, 0.13732653608351372), (20, 0.2168251319235133), (21, 0.2168251319235133), (22, 0.2168251319235133), (23, 0.2168251319235133)]
4 [(9, -0.06878213426735816), (12, 0.06878213426735816), (13, 0.1

'\n    Human machine interface for lab abc computer applications\n    A survey of user opinion of computer system response time\n'

In [33]:
'''
group based on question and then apply bm25
framenet
''' 
fn_docs = 'mycorpus.txt'
raw_data = [['Human', 'machine', 'interface', 'for', 'lab', 'abc', 'computer', 'applications'],
            ['A', 'survey', 'of', 'user', 'opinion', 'of', 'computer', 'system', 'response', 'time'], 
            ['The', 'EPS', 'user', 'interface', 'management', 'system'], 
            ['System', 'and', 'human', 'system', 'engineering', 'testing', 'of', 'EPS'], 
            ['Relation', 'of', 'user', 'perceived', 'response', 'time', 'to', 'error', 'measurement'], 
            ['The', 'generation', 'of', 'random', 'binary', 'unordered', 'trees'], 
            ['The', 'intersection', 'graph', 'of', 'paths', 'in', 'trees'], 
            ['Graph', 'IV', 'Widths', 'of', 'trees', 'and', 'well', 'quasi', 'ordering'],
            ['Graph', 'minors', 'A', 'survey']]
bm25 = BM25(fn_docs, raw_data, delimiter=' ')
print bm25
Query = 'What is a Graph ?'
Query = Query.split()
scores = bm25.BM25Score(Query)
print "Scores",scores




doc ['Human', 'machine', 'interface', 'for', 'lab', 'abc', 'computer', 'applications']
doc ['A', 'survey', 'of', 'user', 'opinion', 'of', 'computer', 'system', 'response', 'time']
doc ['The', 'EPS', 'user', 'interface', 'management', 'system']
doc ['System', 'and', 'human', 'system', 'engineering', 'testing', 'of', 'EPS']
doc ['Relation', 'of', 'user', 'perceived', 'response', 'time', 'to', 'error', 'measurement']
doc ['The', 'generation', 'of', 'random', 'binary', 'unordered', 'trees']
doc ['The', 'intersection', 'graph', 'of', 'paths', 'in', 'trees']
doc ['Graph', 'IV', 'Widths', 'of', 'trees', 'and', 'well', 'quasi', 'ordering']
doc ['Graph', 'minors', 'A', 'survey']
<__main__.BM25 instance at 0x10a331d88>
Scores [0, 0, 0, 0, 0, 0, 0, 0.15791141726235663, 0.5415694380758288]
